<a href="https://colab.research.google.com/github/shivani98patil/Deep-Learning-Algorithms/blob/main/ArtificialNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Artificial Neural Network**

Problem Statement-  We are given data of bank and we have to figure out if if customer will leave the bank or not. 

Features - Customer_id, creditScore, Geography, Gender, Age, Tenure, Balance, NoOfProducts, hasCard, isActiveMember, EstimatedSalary, Exited


Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__


'2.4.1'

### Part 1: Data Pre-Processing

In [ ]:
# Initial features do not have any impact so we are taking feature from credit score till 
#the second last for matrix of feature X
# Last feature for Maxtrix of dependent variable.
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:,3:-1].values 
Y = dataset.iloc[:,-1].values

In [ ]:
print(X)
print(Y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


## Encoding Categorical Data - Gender and Geography

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2, random_state = 0)

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### **Build ANN**

Initializing ANN

In [ ]:
ann = tf.keras.models.Sequential()


Adding Input Layer and First Hidden Layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Adding Second Hidden Layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Adding Output Layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

### **Training ANN**

Compliling ANN

In [ ]:
ann.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

Training ANN

In [ ]:
ann.fit(X_train, Y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6901 - accuracy: 0.5836
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4918 - accuracy: 0.7964
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4607 - accuracy: 0.7983
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4310 - accuracy: 0.8091
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4276 - accuracy: 0.8084
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4138 - accuracy: 0.8194
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3840 - accuracy: 0.8311
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3782 - accuracy: 0.8410
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3618 - accuracy: 0.8462
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3631 - accura

Evaluating the model

Predicting the result of a single observation.
Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

Important note 1: Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

Important note 2: Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### **Predicting the test set results**

In [ ]:
y_pred = ann.predict(X_test)
y_pred = y_pred > 0.5
print(np.concatenate((y_pred.reshape(len(y_pred),1), Y_test.reshape(len(y_pred), 1)),1 ))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, y_pred)
print(cm)
accuracy_score(Y_test, y_pred)

[[1510   85]
 [ 191  214]]


0.862